In [4]:
import requests, time
from bs4 import BeautifulSoup
import pandas as pd

pd.set_option('display.max_columns', None)


sold_house = pd.DataFrame()

# sold_house

In [5]:
base_url = 'https://sh.ke.com/chengjiao/pg' 
house_index = 0
headers = {'User-Agent': 'Mozilla/5.0'}

for url_index in range(1, 101):
    # load page from url
    page = requests.get(base_url + str(url_index), headers = headers)
    soup = BeautifulSoup(page.text, 'html.parser')

    # iterate house landing page lists
    house_list = soup.find('ul', class_='listContent')

    # get each panel of house page
    houses = house_list.find_all('li', {'data-view-event': 'ModuleExpo'})
    
    # get inside each page
    for house in houses:
        
        community_name, house_type, house_area = house.find('a', {'class': 'CLICKDATA maidian-detail'}).getText().split(' ')
        sold_house.at[house_index, '小区名称'] = community_name
        sold_house.at[house_index, '房屋户型'] = house_type
        sold_house.at[house_index, '建筑面积'] = house_area
        
        deal_date = house.find('div', {'class': 'dealDate'}).getText().strip()
        sold_house.at[house_index, '成交日期'] = deal_date
        
        
        house_url = house.find('a', {'class': 'CLICKDATA maidian-detail'}).get('href')
        sold_house.at[house_index, '房屋链接'] = house_url
        
        total_price = house.find('div', {'class': 'totalPrice'}).find('span').getText()
        sold_house.at[house_index, '成交价'] = total_price
        
        price_unit = house.find('div', {'class': 'flood'}).find('span', {'class': 'number'}).getText()
        sold_house.at[house_index, '单价'] = price_unit
        
        cycle_days = house.find('div', {'class': 'dealCycleeInfo'}).find_all('span')[-1].getText().replace('成交周期', '')
        sold_house.at[house_index, '成交周期'] = cycle_days
        
        # open page
        house_page = page = requests.get(house_url, headers = headers)
        print('页面 %d, 已爬房屋 %d, 小区名称: %s, 链接 %s' % (url_index, house_index + 1, community_name, house_url))
        
        house_soup = BeautifulSoup(house_page.text, 'html.parser')
        
        house_bread = house_soup.find('div', {'class': "fl l-txt"}).find_all('a')
        house_bread = [name.getText() for name in house_bread]
        house_region = [name.replace('二手房成交', '') for name in house_bread[-3:]]
        sold_house.at[house_index, '行政区'] = house_region[0]
        sold_house.at[house_index, '区域名称'] = house_region[1]
        
        # 房屋属性
        house_attributes = house_soup.find('div', {'class': 'introContent'}).find_all('li')
        for attribute in house_attributes:
            sold_house.at[house_index, attribute.find('span').getText()] = attribute.find('span').next_sibling.strip()
        
        
        # 房屋特色
        house_feature_section = house_soup.find('div', {'id': 'house_feature'})
        if house_feature_section != None:
            house_feature_name = house_feature_section.find_all('div', {'class': 'name'})
            house_feature_names = [name.getText() for name in house_feature_name]
            house_feature_content = house_feature_section.find_all('div', {'class': 'content'})
            house_feature_contents = [content.getText().strip() for content in house_feature_content]

            for feature_index in range(len(house_feature_names)):
                sold_house.at[house_index, house_feature_names[feature_index]] = house_feature_contents[feature_index]
        
        house_index += 1

页面 1, 已爬房屋 1, 小区名称: 久阳滨江公寓, 链接 https://sh.ke.com/chengjiao/107102526423.html
页面 1, 已爬房屋 2, 小区名称: 天山星城, 链接 https://sh.ke.com/chengjiao/107102549550.html
页面 1, 已爬房屋 3, 小区名称: 中山苑, 链接 https://sh.ke.com/chengjiao/107102503988.html
页面 1, 已爬房屋 4, 小区名称: 紫翠苑, 链接 https://sh.ke.com/chengjiao/107102612644.html
页面 1, 已爬房屋 5, 小区名称: 德州大楼, 链接 https://sh.ke.com/chengjiao/107102373689.html
页面 1, 已爬房屋 6, 小区名称: 月厦新天地, 链接 https://sh.ke.com/chengjiao/107102742425.html
页面 1, 已爬房屋 7, 小区名称: 上南七村, 链接 https://sh.ke.com/chengjiao/107102771999.html
页面 1, 已爬房屋 8, 小区名称: 锦绣华都, 链接 https://sh.ke.com/chengjiao/107102726597.html
页面 1, 已爬房屋 9, 小区名称: 长宁路405弄, 链接 https://sh.ke.com/chengjiao/107102663328.html
页面 1, 已爬房屋 10, 小区名称: 中浩云花园, 链接 https://sh.ke.com/chengjiao/107101747931.html
页面 1, 已爬房屋 11, 小区名称: 由由四村, 链接 https://sh.ke.com/chengjiao/107102735476.html
页面 1, 已爬房屋 12, 小区名称: 菊泉新城(陆翔路678弄), 链接 https://sh.ke.com/chengjiao/107102638885.html
页面 1, 已爬房屋 13, 小区名称: 芳雅苑, 链接 https://sh.ke.com/chengjiao/107102075521.html
页面 1, 已爬

页面 4, 已爬房屋 107, 小区名称: 中邦城市, 链接 https://sh.ke.com/chengjiao/107101663467.html
页面 4, 已爬房屋 108, 小区名称: 蓬莱苑, 链接 https://sh.ke.com/chengjiao/107102671145.html
页面 4, 已爬房屋 109, 小区名称: 文华苑(宝山), 链接 https://sh.ke.com/chengjiao/107102606512.html
页面 4, 已爬房屋 110, 小区名称: 田林三村, 链接 https://sh.ke.com/chengjiao/107102722514.html
页面 4, 已爬房屋 111, 小区名称: 银河新都, 链接 https://sh.ke.com/chengjiao/107102554673.html
页面 4, 已爬房屋 112, 小区名称: 韵都城, 链接 https://sh.ke.com/chengjiao/107102455080.html
页面 4, 已爬房屋 113, 小区名称: 东兴小区, 链接 https://sh.ke.com/chengjiao/107102753346.html
页面 4, 已爬房屋 114, 小区名称: 紫薇茗庭, 链接 https://sh.ke.com/chengjiao/107102215039.html
页面 4, 已爬房屋 115, 小区名称: 崧泽华城逸泰雅苑, 链接 https://sh.ke.com/chengjiao/107101398708.html
页面 4, 已爬房屋 116, 小区名称: 紫薇茗庭, 链接 https://sh.ke.com/chengjiao/107102675407.html
页面 4, 已爬房屋 117, 小区名称: 凤城六村, 链接 https://sh.ke.com/chengjiao/107101561270.html
页面 4, 已爬房屋 118, 小区名称: 南泉小区, 链接 https://sh.ke.com/chengjiao/107102756197.html
页面 4, 已爬房屋 119, 小区名称: 新海小区(长宁), 链接 https://sh.ke.com/chengjiao/10710263

页面 8, 已爬房屋 212, 小区名称: 大发融悦, 链接 https://sh.ke.com/chengjiao/107102118451.html
页面 8, 已爬房屋 213, 小区名称: 舒乐小区, 链接 https://sh.ke.com/chengjiao/107102367965.html
页面 8, 已爬房屋 214, 小区名称: 梅园三村, 链接 https://sh.ke.com/chengjiao/107102539376.html
页面 8, 已爬房屋 215, 小区名称: 虹梅南路126弄, 链接 https://sh.ke.com/chengjiao/107102389278.html
页面 8, 已爬房屋 216, 小区名称: 万科阳光苑, 链接 https://sh.ke.com/chengjiao/107102477240.html
页面 8, 已爬房屋 217, 小区名称: 瑞冬小区, 链接 https://sh.ke.com/chengjiao/107102646434.html
页面 8, 已爬房屋 218, 小区名称: 金丰佳园, 链接 https://sh.ke.com/chengjiao/107102113001.html
页面 8, 已爬房屋 219, 小区名称: 育秀六区, 链接 https://sh.ke.com/chengjiao/107102062014.html
页面 8, 已爬房屋 220, 小区名称: 万荣小区, 链接 https://sh.ke.com/chengjiao/107102038916.html
页面 8, 已爬房屋 221, 小区名称: 海洲丽园, 链接 https://sh.ke.com/chengjiao/107102652539.html
页面 8, 已爬房屋 222, 小区名称: 新舒苑, 链接 https://sh.ke.com/chengjiao/107102713721.html
页面 8, 已爬房屋 223, 小区名称: 紫晶苑, 链接 https://sh.ke.com/chengjiao/107102729313.html
页面 8, 已爬房屋 224, 小区名称: 公园3000黄兴花园, 链接 https://sh.ke.com/chengjiao/10710209

页面 11, 已爬房屋 316, 小区名称: 新湖青蓝国际, 链接 https://sh.ke.com/chengjiao/107002330159.html
页面 11, 已爬房屋 317, 小区名称: 圣骊澳门苑, 链接 https://sh.ke.com/chengjiao/107102541730.html
页面 11, 已爬房屋 318, 小区名称: 中芯花园199弄, 链接 https://sh.ke.com/chengjiao/107102671245.html
页面 11, 已爬房屋 319, 小区名称: 陈家宅小区(潍坊西路), 链接 https://sh.ke.com/chengjiao/107102487450.html
页面 11, 已爬房屋 320, 小区名称: 飞碟苑, 链接 https://sh.ke.com/chengjiao/107102761878.html
页面 11, 已爬房屋 321, 小区名称: 中恒苑, 链接 https://sh.ke.com/chengjiao/107102766943.html
页面 11, 已爬房屋 322, 小区名称: 康平小区, 链接 https://sh.ke.com/chengjiao/107102576321.html
页面 11, 已爬房屋 323, 小区名称: 龙柏西郊公寓, 链接 https://sh.ke.com/chengjiao/107102772585.html
页面 11, 已爬房屋 324, 小区名称: 罗南二村, 链接 https://sh.ke.com/chengjiao/107102147998.html
页面 11, 已爬房屋 325, 小区名称: 育秀六区, 链接 https://sh.ke.com/chengjiao/107102755116.html
页面 11, 已爬房屋 326, 小区名称: 复兴佳苑, 链接 https://sh.ke.com/chengjiao/107101957398.html
页面 11, 已爬房屋 327, 小区名称: 银都新村(1至7区), 链接 https://sh.ke.com/chengjiao/107102770382.html
页面 11, 已爬房屋 328, 小区名称: 八一小区, 链接 https://sh.k

页面 14, 已爬房屋 420, 小区名称: 石泉小区, 链接 https://sh.ke.com/chengjiao/107102356004.html
页面 15, 已爬房屋 421, 小区名称: 云都苑, 链接 https://sh.ke.com/chengjiao/107102679560.html
页面 15, 已爬房屋 422, 小区名称: 古美小区, 链接 https://sh.ke.com/chengjiao/107102551906.html
页面 15, 已爬房屋 423, 小区名称: 茅台新苑, 链接 https://sh.ke.com/chengjiao/107102313391.html
页面 15, 已爬房屋 424, 小区名称: 泰东新村(徐汇), 链接 https://sh.ke.com/chengjiao/107102479462.html
页面 15, 已爬房屋 425, 小区名称: 临沂三村, 链接 https://sh.ke.com/chengjiao/107102725236.html
页面 15, 已爬房屋 426, 小区名称: 海棠馨苑南区, 链接 https://sh.ke.com/chengjiao/107102659806.html
页面 15, 已爬房屋 427, 小区名称: 玉兰一村, 链接 https://sh.ke.com/chengjiao/107102522079.html
页面 15, 已爬房屋 428, 小区名称: 新闸路1213-1229号(单), 链接 https://sh.ke.com/chengjiao/107102373981.html
页面 15, 已爬房屋 429, 小区名称: 浦江智汇园, 链接 https://sh.ke.com/chengjiao/107102525533.html
页面 15, 已爬房屋 430, 小区名称: 益民小区, 链接 https://sh.ke.com/chengjiao/107102286736.html
页面 15, 已爬房屋 431, 小区名称: 佳宝新村, 链接 https://sh.ke.com/chengjiao/107102721431.html
页面 15, 已爬房屋 432, 小区名称: 金汇华光城, 链接 https://sh.ke

...

页面 86, 已爬房屋 2573, 小区名称: 罗阳四村, 链接 https://sh.ke.com/chengjiao/107102545862.html
页面 86, 已爬房屋 2574, 小区名称: 共富二村, 链接 https://sh.ke.com/chengjiao/107102205404.html
页面 86, 已爬房屋 2575, 小区名称: 汉中小区, 链接 https://sh.ke.com/chengjiao/107101953631.html
页面 86, 已爬房屋 2576, 小区名称: 新一小区, 链接 https://sh.ke.com/chengjiao/107102519899.html
页面 86, 已爬房屋 2577, 小区名称: 中辉新苑, 链接 https://sh.ke.com/chengjiao/107102519888.html
页面 86, 已爬房屋 2578, 小区名称: 西上海御庭, 链接 https://sh.ke.com/chengjiao/107102466626.html
页面 86, 已爬房屋 2579, 小区名称: 嘉乐公寓, 链接 https://sh.ke.com/chengjiao/107100966795.html
页面 86, 已爬房屋 2580, 小区名称: 崧泽华城华中苑, 链接 https://sh.ke.com/chengjiao/107102654891.html
页面 87, 已爬房屋 2581, 小区名称: 鞍山四村第二小区, 链接 https://sh.ke.com/chengjiao/107102589900.html
页面 87, 已爬房屋 2582, 小区名称: 绿地香颂(奉贤)(别墅), 链接 https://sh.ke.com/chengjiao/107101279429.html
页面 87, 已爬房屋 2583, 小区名称: 万科清林径, 链接 https://sh.ke.com/chengjiao/107102571397.html
页面 87, 已爬房屋 2584, 小区名称: 番禺小区, 链接 https://sh.ke.com/chengjiao/107102704322.html
页面 87, 已爬房屋 2585, 小区名称: 首创锦悦, 链接 ht

页面 90, 已爬房屋 2676, 小区名称: 惠康苑, 链接 https://sh.ke.com/chengjiao/107102133442.html
页面 90, 已爬房屋 2677, 小区名称: 华泾五村, 链接 https://sh.ke.com/chengjiao/107102489574.html
页面 90, 已爬房屋 2678, 小区名称: 英郡别院, 链接 https://sh.ke.com/chengjiao/107102022650.html
页面 90, 已爬房屋 2679, 小区名称: 建国西路212号, 链接 https://sh.ke.com/chengjiao/107102676512.html
页面 90, 已爬房屋 2680, 小区名称: 景瑞城中公园, 链接 https://sh.ke.com/chengjiao/107102204376.html
页面 90, 已爬房屋 2681, 小区名称: 浦江世博家园八街坊, 链接 https://sh.ke.com/chengjiao/107102668699.html
页面 90, 已爬房屋 2682, 小区名称: 嘉善坊, 链接 https://sh.ke.com/chengjiao/107102636809.html
页面 90, 已爬房屋 2683, 小区名称: 飞虹北村, 链接 https://sh.ke.com/chengjiao/107102147561.html
页面 90, 已爬房屋 2684, 小区名称: 安阳小区, 链接 https://sh.ke.com/chengjiao/107102629064.html
页面 90, 已爬房屋 2685, 小区名称: 控江路1209号, 链接 https://sh.ke.com/chengjiao/107102244810.html
页面 90, 已爬房屋 2686, 小区名称: 竹园新村(浦东), 链接 https://sh.ke.com/chengjiao/107102699429.html
页面 90, 已爬房屋 2687, 小区名称: 恒大华城新华苑, 链接 https://sh.ke.com/chengjiao/107102034597.html
页面 90, 已爬房屋 2688, 小区名称: 同润菲诗艾伦, 

页面 93, 已爬房屋 2778, 小区名称: 江海二村(奉贤), 链接 https://sh.ke.com/chengjiao/107102591154.html
页面 93, 已爬房屋 2779, 小区名称: 黄陵小区, 链接 https://sh.ke.com/chengjiao/107102672101.html
页面 93, 已爬房屋 2780, 小区名称: 大上海国际花园八区, 链接 https://sh.ke.com/chengjiao/107102524597.html
页面 93, 已爬房屋 2781, 小区名称: 万荣小区, 链接 https://sh.ke.com/chengjiao/107102100221.html
页面 93, 已爬房屋 2782, 小区名称: 三林安居苑, 链接 https://sh.ke.com/chengjiao/107102439715.html
页面 93, 已爬房屋 2783, 小区名称: 江海新村, 链接 https://sh.ke.com/chengjiao/107102678405.html
页面 93, 已爬房屋 2784, 小区名称: 龙柏井亭公寓, 链接 https://sh.ke.com/chengjiao/107102638924.html
页面 93, 已爬房屋 2785, 小区名称: 阳光里(嘉定), 链接 https://sh.ke.com/chengjiao/107102702250.html
页面 93, 已爬房屋 2786, 小区名称: 新海小区(长宁), 链接 https://sh.ke.com/chengjiao/107102652284.html
页面 93, 已爬房屋 2787, 小区名称: 宝钢九村, 链接 https://sh.ke.com/chengjiao/107101612570.html
页面 93, 已爬房屋 2788, 小区名称: 莘松三村, 链接 https://sh.ke.com/chengjiao/107102567414.html
页面 93, 已爬房屋 2789, 小区名称: 平顺小区, 链接 https://sh.ke.com/chengjiao/107102678006.html
页面 93, 已爬房屋 2790, 小区名称: 古南小区, 链接 

页面 96, 已爬房屋 2880, 小区名称: 斜土路723弄, 链接 https://sh.ke.com/chengjiao/107102453476.html
页面 97, 已爬房屋 2881, 小区名称: 嘉邦小区, 链接 https://sh.ke.com/chengjiao/107102687221.html
页面 97, 已爬房屋 2882, 小区名称: 欣春苑, 链接 https://sh.ke.com/chengjiao/107102401003.html
页面 97, 已爬房屋 2883, 小区名称: 绿园一村, 链接 https://sh.ke.com/chengjiao/107102337857.html
页面 97, 已爬房屋 2884, 小区名称: 碧云新天地(一期), 链接 https://sh.ke.com/chengjiao/107102433585.html
页面 97, 已爬房屋 2885, 小区名称: 绿地玲珑寓, 链接 https://sh.ke.com/chengjiao/107101911538.html
页面 97, 已爬房屋 2886, 小区名称: 名门河滨花园, 链接 https://sh.ke.com/chengjiao/107102464832.html
页面 97, 已爬房屋 2887, 小区名称: 同济新村, 链接 https://sh.ke.com/chengjiao/107102679874.html
页面 97, 已爬房屋 2888, 小区名称: 白玉新村, 链接 https://sh.ke.com/chengjiao/107101713868.html
页面 97, 已爬房屋 2889, 小区名称: 陆家嘴红醍半岛(别墅), 链接 https://sh.ke.com/chengjiao/107102681370.html
页面 97, 已爬房屋 2890, 小区名称: 古北新城, 链接 https://sh.ke.com/chengjiao/107102694440.html
页面 97, 已爬房屋 2891, 小区名称: 绿川公寓, 链接 https://sh.ke.com/chengjiao/107102569871.html
页面 97, 已爬房屋 2892, 小区名称: 新安小区, 链接 ht

页面 100, 已爬房屋 2982, 小区名称: 延长小区(普陀), 链接 https://sh.ke.com/chengjiao/107102675565.html
页面 100, 已爬房屋 2983, 小区名称: 月浦九村, 链接 https://sh.ke.com/chengjiao/107102490660.html
页面 100, 已爬房屋 2984, 小区名称: 银龙小区, 链接 https://sh.ke.com/chengjiao/107101995096.html
页面 100, 已爬房屋 2985, 小区名称: 永泰花苑, 链接 https://sh.ke.com/chengjiao/107102589954.html
页面 100, 已爬房屋 2986, 小区名称: 静安新城一区, 链接 https://sh.ke.com/chengjiao/107102256779.html
页面 100, 已爬房屋 2987, 小区名称: 万科优诗美地, 链接 https://sh.ke.com/chengjiao/107100855814.html
页面 100, 已爬房屋 2988, 小区名称: 华光花园, 链接 https://sh.ke.com/chengjiao/107102255403.html
页面 100, 已爬房屋 2989, 小区名称: 月厦新天地, 链接 https://sh.ke.com/chengjiao/107102663987.html
页面 100, 已爬房屋 2990, 小区名称: 富友嘉园, 链接 https://sh.ke.com/chengjiao/107102667514.html
页面 100, 已爬房屋 2991, 小区名称: 上大阳光乾静园, 链接 https://sh.ke.com/chengjiao/107102642877.html
页面 100, 已爬房屋 2992, 小区名称: 九城湖滨国际, 链接 https://sh.ke.com/chengjiao/107101711539.html
页面 100, 已爬房屋 2993, 小区名称: 古龙苑, 链接 https://sh.ke.com/chengjiao/107102694098.html
页面 100, 已爬房屋 2994, 小区名称: 乾溪

In [6]:
sold_house.to_excel('上海成交房产信息.xlsx', index=False)

In [73]:
amap_key = '1f6202af0ebf2197d6524c9dfd18c470'
start = 2750
end = 3000

city_name = '上海'
house_index = start

for index, row in sold_house[start: end].iterrows():
    
    address = row['小区名称'] + ' ' + row['行政区'] + ' ' + row['区域名称']

    # get geo location (latitude, longitude)
    url = 'http://restapi.amap.com/v3/geocode/geo?key=' + amap_key + '&address=' + address + '&city=' + city_name
    page = requests.get(url)
    geo_response = page.json()
    
    # location may be empty
    if geo_response['geocodes'] != []:
        location = geo_response['geocodes'][0]['location']
    else:
        address = row['小区名称']

        # get geo location (latitude, longitude)
        url = 'http://restapi.amap.com/v3/geocode/geo?key=' + amap_key + '&address=' + address + '&city=' + city_name
        page = requests.get(url)
        geo_response = page.json()
        

    # assign latitude and longitude
    latitude, longitude = location.split(',')
    print(index, address)
    
    # 最近距离
    keywords = ['地铁', '火车站', '机场', '公园']
    for keyword in keywords:
        url = 'http://restapi.amap.com/v3/place/around?key=' + amap_key + '&location=' + location + '&keywords=' + keyword + '&types=&offset=&page=&extensions=all'
        field_page = requests.get(url)
        place_response = field_page.json()
        name = place_response['pois'][0]['name']
        distance = place_response['pois'][0]['distance']
        sold_house.at[house_index, '最近' + keyword] = name
        sold_house.at[house_index, '最近' + keyword + '距离'] = distance
    
    # 1000米内设置个数
    keywords = ['小学', '初中', '医院', '商场']

    for keyword in keywords:
        url = 'http://restapi.amap.com/v3/place/around?key=' + amap_key + '&location=' + location + '&keywords=' + keyword + '&types=&offset=&page=&extensions=all'
        field_page = requests.get(url)
        place_response = field_page.json()
        location_feedback = place_response['pois']
        location_number = 0
        for location_index in range(len(location_feedback)):
            location_name = location_feedback[location_index]['name']
            location_distance = location_feedback[location_index]['distance']

            if int(location_distance) <= 1000:
                # print(location_name, location_distance)
                location_number += 1
        #print(location_number)
        sold_house.at[house_index, '1Km' + keyword + '个数'] = str(location_number)
    
    house_index+= 1


2750 柳营路1025弄 静安 大宁
2751 中邦大都会 浦东 康桥
2752 长宁馥邦苑 长宁 北新泾
2753 南苑二村 嘉定 嘉定老城
2754 殷高小区 宝山 高境
2755 共富花园 宝山 共富
2756 禹洲蓝爵 浦东 金桥
2757 平顺小区 静安 彭浦
2758 鸿宝一村(南区) 奉贤 西渡
2759 上钢七村 浦东 世博
2760 新高苑(一期) 浦东 高行
2761 永福第六小区 徐汇 衡山路
2762 新高苑(二期) 浦东 高行
2763 招商海德名门 宝山 杨行
2764 凤城三村 杨浦 控江路
2765 新凯城锦昌苑 松江 泗泾
2766 岭南路700弄 静安 彭浦
2767 新凯城香樟苑 松江 泗泾
2768 航华一村 闵行 航华
2769 合景理想家 嘉定 嘉定新城
2770 中信和平家园 虹口 临平路
2771 天环苑 浦东 川沙
2772 御桥花园 浦东 御桥
2773 梅陇六村 徐汇 华东理工
2774 大华锦绣华城(十八街区)(公寓) 浦东 北蔡
2775 名门世家(浦东) 浦东 洋泾
2776 红莲小区 浦东 世博
2777 江海二村(奉贤) 奉贤 南桥
2778 黄陵小区 普陀 甘泉宜川
2779 大上海国际花园八区 闵行 静安新城
2780 万荣小区 静安 阳城
2781 三林安居苑 浦东 三林
2782 江海新村 奉贤 南桥
2783 龙柏井亭公寓 闵行 龙柏
2784 阳光里(嘉定) 嘉定 嘉定新城
2785 新海小区(长宁) 长宁 镇宁路
2786 宝钢九村 宝山 淞宝
2787 莘松三村 闵行 莘庄
2788 平顺小区 静安 彭浦
2789 古南小区 长宁 天山
2790 孙桥花苑 浦东 张江
2791 三湘四季花城 松江 松江大学城
2792 南苑五村 嘉定 嘉定老城
2793 大华二村 宝山 大华
2794 凤凰景苑 闵行 老闵行
2795 长桥一村 徐汇 长桥
2796 浦发绿城永泰路2079弄 浦东 三林
2797 花苑村紫竹园 徐汇 康健
2798 知雅汇(公寓) 松江 九亭
2799 丽园路1032弄 黄浦 打浦桥
2800 康泰新城 宝山 大华
2801 平阳一村 闵行 古美
2802 周家湾小区 徐汇 龙华
2803 潍坊十村 浦东 潍坊
2804 保利星海屿筑 嘉定 嘉定新城
2805 新领地

In [74]:
sold_house

,小区名称,房屋户型,建筑面积,成交日期,房屋链接,成交价,单价,成交周期,行政区,区域名称,所在楼层,户型结构,套内面积,建筑类型,房屋朝向,建成年代,装修情况,建筑结构,供暖方式,梯户比例,配备电梯,链家编号,交易权属,挂牌时间,房屋用途,房屋年限,房权所属,房源标签,核心卖点,小区介绍,适宜人群,户型介绍,周边配套,交通出行,税费解析,售房详情,装修描述,权属抵押,最近地铁,最近地铁距离,最近火车站,最近火车站距离,最近机场,最近机场距离,最近公园,最近公园距离,1Km小学个数,1Km初中个数,1Km医院个数,1Km商场个数
0,久阳滨江公寓,2室1厅1厨1卫,78.25㎡,2020.07.10,https://sh.ke.com/chengjiao/107102526423.html,528,67477,59天,浦东,陆家嘴,高楼层 (共25层),复式,暂无数据,板楼,南,2006,精装,钢混结构,暂无数据,四梯二十二户,有,107102526423,商品房,2020-05-13,普通住宅,暂无数据,非共有,地铁,业主诚意，景观佳，看陆家嘴，酒店管理，有5星酒店感觉,此房是高区的，复式结构，两种户型 1 楼下客厅朝南，楼上主卧朝北 看陆家嘴金融区夜景！！ 2...,适宜新上海群体，置换人群，年轻人群，小区安静，生活方便，购物，吃饭，出行都方便,久阳2房户型，客厅朝南，卧室朝南，楼下客厅 餐厅，楼上卧室，动静分离，私密性好,商场：国金广场、正大广场、八佰伴、上海湾、华润时代广场； 餐饮：诺莱仕游艇会、浅深、新意尚；...,地铁2号线陆家嘴站、东昌路站；公交：451路崂山路张杨路站，789路崂山路张杨路站，86路，...,NaN,NaN,NaN,NaN,商城路(地铁站),896,上海虹桥站,19028,上海浦东国际机场,29153,老白渡滨江绿地,658,8.0,0.0,5.0,19.0
1,天山星城,1室1厅1厨1卫,50.74㎡,2020.07.10,https://sh.ke.com/chengjiao/107102549550.html,517,101892,54天,长宁,天山,高楼层 (共13层),平层,暂无数据,板楼,南,2006,精装,钢混结构,暂无数据,一梯四户,有,107102549550,商品房,2020-05-18,普通住宅,暂无数据,非共有,地铁,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,威宁路(地铁站),183,上海虹桥站,6725,上海浦东国际机场,40928,天原公园,371,3.0,3.0,5.0,5.0
2,中山苑,3室2厅1厨2卫,127.54㎡,2020.07.10,https://sh.ke.com/chengjiao/107102503988.html,1050,82328,63天,徐汇,龙华,高楼层 (共6层),平层,暂无数据,板楼,南,1994,精装,砖混结构,暂无数据,一梯两户,有,107102503988,商品房,2020-05-09,普通住宅,暂无数据,共有,地铁,600米内三条地铁，两南一北，电梯房，外销房，阳台不算面积,NaN,配套齐全完善，生活便利，适合自住和陪读的家庭（上学近），旁边就是菜市场、饭店和熟食店，超市、...,经典板式户型，三房两厅两卫，两南一北，厅通透，采光无遮挡，朝南带大阳台,旁边有龙山幼儿园（市一级）；万体馆、星游城、飞洲国际、美罗城；双峰菜市场、朵云轩、东安公园、...,地铁4号线上海体育场站，11号线上海游泳馆站，还有1号线上海体育馆站，3、12号线龙漕路站，...,NaN,NaN,NaN,NaN,上海游泳馆(地铁站),555,上海虹桥站,12153,上海浦东国际机场,34740,憩园,670,4.0,0.0,13.0,6.0
3,紫翠苑,1室1厅1厨1卫,56.03㎡,2020.07.10,https://sh.ke.com/chengjiao/107102612644.html,256,45601,40天,浦东,高行,低楼层 (共6层),平层,暂无数据,板楼,南,2007,精装,钢混结构,暂无数据,一梯两户,无,107102612644,动迁安置房,2020-06-01,普通住宅,暂无数据,共有,NaN,07年小区，位置好，前后无遮挡，税费少，诚心出售！,生活气息浓郁，紧靠小区东边就是农工商超市，俱进菜市场，还有万嘉购物广场，沃尔玛大型超市，紧靠...,NaN,户型是客厅朝南带大阳台，南北通，户型方正明亮，业主自住装修温馨！每个房间都有窗户，全明。,NaN,有774路、843路、406路、外高桥4路等公交车，距离12号线金京路站2.7公里，公共交通便捷。,房子是商品房，为普通住宅，没有增值税。满五年不唯一，有一个点的个税。如果您是首套需要缴纳一个...,NaN,NaN,NaN,五洲大道(地铁站),2296,上海虹桥站,30383,上海浦东国际机场,25718,新高苑南侧休闲公园,1507,1.0,0.0,1.0,0.0
4,德州大楼,2室1厅1厨1卫,81.28㎡,2020.07.10,https://sh.ke.com/chengjiao/107102373689.html,408,50197,91天,浦东,世博,低楼层 (共25层),平层,暂无数据,塔楼,南,1992,精装,钢混结构,暂无数据,两梯九户,有,107102373689,商品房,2020-04-11,普通住宅,暂无数据,非共有,地铁,此房一手动迁，满五年唯一无增值税，厨卫朝北全明精装两房一厅。,NaN,本房是业主婚房，装修用料好，可拎包入住的精装修，电梯房适合老幼孕妇，因距离地铁近也很适合上班...,房子是1992年房龄，两梯九户，厨房卫生间朝北，卧室一间朝南一间朝北，客厅朝南。户型南北通风...,NaN,交通：近地铁8/13号成山路地铁站3号口 \n生活：上南公园可供老人小孩休息玩耍 ，上钢菜场...,NaN,此房是业主的婚房装修，现在有了小孩想换个大房子住，看房提前约，业主诚意售房。,本房是业主婚房，装修用料好，可拎包入住的精装修，自住保养好，看房提前约。,NaN,成山路(地铁站),517,上海虹桥站,16687,上海浦东国际机场,30250,上南公园,579,9.0,0.0,9.0,3.0
5,月厦新天地,1室1厅1厨1卫,50.75㎡,2020.07.10,https://sh.ke.com/chengjiao/107102742425.html,96,18818,11天,松江,松江老城,低楼层 (共18层),暂无数据,暂无数据,塔楼,西,2005,简装,钢混结构,暂无数据,三梯三十七户,有,107102742425,商品房,2020-06-30,普通住宅,暂无数据,暂无数据,NaN,满五 电梯房 配套设施齐全 交通便利,小区位于松江老城板块。2005年竣工的，为数不多的电梯房。小区环境好,NaN,NaN,有交通枢纽，9号线体育中xi站。距离1公里处，有大型的购物中xi，沃尔玛超市和卜蜂莲花。,距离9号线体育中xin站步行，一会可到，门口有公交，出门十分方便。,NaN,NaN,NaN,NaN,松江体育中心(地铁站),953,上海虹桥站,21881,上海浦东国际机场,56350,菜花泾绿地,86,3.0,4.0,3.0,9.0
6,上南七村,2室1厅1厨1卫,49.15㎡,2020.07.10,https://sh.ke.com/chengjiao/107102771999.html,260,52900,5天,浦东,世博,高楼层 (共6层),平层,暂无数据,板楼,南,1986,精装,砖混结构,暂无数据,一梯四户,无,107102771999,商品房,2020-07-06,普通住宅,暂无数据,非共有,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,东明路(地铁站),754,上海虹桥站,17572,上海浦东国际机场,29322,蔓趣公园,259,11.0,0.0,9.0,2.0
7,锦绣华都,2室2厅0厨1卫,90.22㎡,2020.07.10,https://sh.ke.com/chengjiao/107102726597.html,485,53758,14天,浦东,御桥,低楼层 (共18层),平层,暂无数据,板楼,南,2010,精装,钢混结构,暂无数据,两梯三户,有,107102726597,商品房,2020-06-27,普通住宅,暂无数据,共有,NaN,景观漂亮 采光好 产权清晰 11-18双地铁1公里 两梯三户 诚意卖,小区是10、14年毛坯分批交付，物业是康万物业，开发商是越州置业，物业费1.75元每平方，地...,小区环境优美，绿化高 交通便利，买这个小区的主要客户群体是：1.张江、陆家嘴一带……上班的年...,NaN,小区500米有社区医院，旁边就是菜市场，在建的大卖场，1公里处有万科2049商圈，万科复地活...,本小区位于中外环之间，地铁11号线18号线御桥站点1公里，16号线罗山路站点到中环1.7公里...,NaN,业主因为置换大房子，新房子已经买好了，产权夫妻2人 很诚意出售，看房很方便,NaN,NaN,御桥(地铁站),1040,上海虹桥站,25224,上海浦东国际机场,21750,幻想公园,555,0.0,0.0,0.0,1.0
8,长宁路405弄,1室0厅1厨1卫,26.14㎡,2020.07.10,https://sh.ke.com/chengjiao/107102663328.html

In [75]:
sold_house.to_excel('上海成交房产信息_动态.xlsx', index=False)